# Рекомендация тарифов

В наличие имеются данные о поведении клиентов, использующие данные тарифы. Необходимо построить модель для задачи классификации, которая выберет подходящий тариф. 

Необходимое значение параметра *accuracy* составляет 0,75.

## Изучение данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

In [2]:
data = pd.read_csv('/datasets/users_behavior.csv')
display(data)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
data.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


### Вывод
Представленные данные изучены, предобработка им не требуется.

## Разбивка данных на выборки

In [5]:
features = data.drop(['is_ultra'], axis =1)
target = data['is_ultra']
features_a, features_valid, target_a, target_valid = train_test_split(features, target, test_size=0.2, random_state=12345)
features_train, features_test, target_train, target_test = train_test_split(features_a, target_a, test_size=0.25, random_state=12345)

In [6]:
print(features_valid.shape)
print(features_train.shape)
print(features_test.shape)

(643, 4)
(1928, 4)
(643, 4)


### Вывод
Данные разбиты на обучающую, валидационную и тестовые выборки в соотношении 3:1:1

## Исследование модели

### Модель решающего дерева

In [7]:
best_model = None
best_result = 0
best_depth = 0
for depth in range(1, 12):
    model = DecisionTreeClassifier(random_state=12345, max_depth= depth) 
    model.fit(features_train, target_train) 
    predictions_valid = model.predict(features_valid) 
    result = accuracy_score(target_valid, predictions_valid) 
    if result > best_result:
        best_model = model
        best_result = result
        best_depth = depth
print('Accuracy наилучшей модели на валидационной выборке:', best_result)
print('Значение параметра max_depth для модели с лучшей Accuracy:', best_depth)

Accuracy наилучшей модели на валидационной выборке: 0.7884914463452566
Значение параметра max_depth для модели с лучшей Accuracy: 5


### Модель случайного леса

In [13]:
best_model = None
best_result = 0
best_est = 0
for est in range(1, 30):
    model = RandomForestClassifier(random_state=12345, n_estimators=est) 
    model.fit(features_train, target_train) 
    result = model.score(features_valid, target_valid) 
    if result > best_result:
        best_model = model
        best_result = result
        best_est = est
print('Accuracy наилучшей модели на валидационной выборке:', best_result)
print('Значение параметра n_estimators для модели с лучшей Accuracy:', best_est)

Accuracy наилучшей модели на валидационной выборке: 0.7916018662519441
Значение параметра n_estimators для модели с лучшей Accuracy: 13


### Модель логистической регрессии

In [9]:
model = LogisticRegression(random_state=12345) 
model.fit(features_train, target_train) 
result = model.score(features_valid, target_valid)
print("Accuracy модели логистической регрессии на валидационной выборке:", result)

Accuracy модели логистической регрессии на валидационной выборке: 0.702954898911353


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


### Вывод
Были проверены модели решающего дерева, случайного леса и логистической регрессии. Проведены исследования на точность модели при смене гиперпараметров(max depth для решающего дерева и n_estimators для случайного леса). По итогам наибольшей точностью обладает модель случайного леса со значением параметра n_estimators равном 13. По общим результатом исследования можно сделать вывод о том что точность первых двух моделей примерно одинакова и превосходит точность модели логистической регрессии. 

## Проверьте модель на тестовой выборке

In [10]:
model = RandomForestClassifier(random_state=12345, n_estimators=13) 
model.fit(features_train, target_train) 
result = model.score(features_test, target_test)
print("Accuracy модели случайного леса на тестовой выборке:", result)                     

Accuracy модели случайного леса на тестовой выборке: 0.7776049766718507


In [11]:
model = DecisionTreeClassifier(random_state=12345, max_depth= 5) 
model.fit(features_train, target_train) 
result = model.score(features_test, target_test)
print("Accuracy модели решающего дерева на тестовой выборке:", result)  

Accuracy модели решающего дерева на тестовой выборке: 0.7589424572317263


### Вывод
Модель с наилучшим значением доли правильных ответов при исследовании с валидационной выборкой была применена на тестовой выборке. Полученное значение accurace составило 0,78 что больше требуемой 0,75, т.е. можно сделать вывод о некоторой успешности применения модели.

## Проверка модели на адекватность

In [12]:
model = DummyClassifier(strategy="most_frequent")
model.fit(features_train, target_train) 
result = model.score(features_test, target_test)
print("Accuracy тривиальной модели на тестовой выборке:", result)        

Accuracy тривиальной модели на тестовой выборке: 0.6889580093312597


### Вывод
При сравнении с тривиальной моделью(в данном случае с моделью предсказывающей для всех случаев более часто встречаемый тариф в обучающей выборке) можно сделать вывод о большей доли правильных ответов(0,78 и 0,69) в выбранной нами модели.